## 数据分析

In [1]:
# 导入包
import time
from datetime import datetime
import pandas as pd
import json
import math

In [2]:
# 弹幕
brg_df = pd.read_csv('../barrages/1818黄金眼弹幕_1-14.csv')
brg_list = []
for i in range(1,15):
    brg_list += list(brg_df[str(i)])

# 5集的评论
cmt_df1 = pd.read_csv('../comments/1818-1.csv')
cmt_df2 = pd.read_csv('../comments/1818-2.csv')
cmt_df3 = pd.read_csv('../comments/1818-3.csv')
cmt_df4 = pd.read_csv('../comments/1818-4.csv')
cmt_df5 = pd.read_csv('../comments/1818-5.csv')


# 总的评论
cmt_df = pd.concat([cmt_df1,cmt_df2,cmt_df3,cmt_df4,cmt_df5],axis=0)

cmt_df = cmt_df.dropna()

# 时间戳转日期
cmt_df['time'] = cmt_df['ctime'].map(lambda x:time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(x)))

# 获取天数
cmt_df['day'] = cmt_df['time'].map(lambda x:x.split(' ')[0])

#获取月份数
cmt_df['month']  = cmt_df['time'].map(lambda x:x.split(' ')[0][:7])

# 得到星期数
cmt_df['week'] = cmt_df['ctime'].map(lambda x:time.localtime(x).tm_wday)
# 得到24小时
cmt_df['hour'] = cmt_df['ctime'].map(lambda x:time.localtime(x).tm_hour)

# 重新设置索引
cmt_df.index = [i for i in range(len(cmt_df))]

cmt_df

,name,level,sex,ctime,rcount,like,content,time,day,month,week,hour
0,maglorthesinger,5.0,女,1.547896e+09,28.0,12962.0,传统媒体拥抱年轻人的成功案例(°∀°)ﾉ,2019-01-19 19:11:42,2019-01-19,2019-01,5,19
1,紗棠糖糖糖,6.0,女,1.547896e+09,16.0,4951.0,电视在放哈哈哈哈哈哈天嘞,2019-01-19 19:04:39,2019-01-19,2019-01,5,19
2,橙橙橙橙橙子,6.0,女,1.547896e+09,8.0,3143.0,配音小哥笑场了！扣钱！[蛆音娘_滑稽],2019-01-19 19:11:44,2019-01-19,2019-01,5,19
3,共青团中央,6.0,保密,1.547903e+09,43.0,2155.0,我的快乐源泉[2233娘_喝水],2019-01-19 21:08:46,2019-01-19,2019-01,5,21
4,吃素的狮子,6.0,男,1.547971e+09,44.0,1668.0,好想上一次1818黄金眼！,2019-01-20 15:49:13,2019-01-20,2019-01,6,15
...,...,...,...,...,...,...,...,...,...,...,...,...
1990,Loverainmaker,6.0,男,1.550319e+09,0.0,0.0,秀的我头皮发麻,2019-02-16 20:12:59,2019-02-16,2019-02,5,20
1991,高坚果Official,5.0,保密,1.550320e+09,0.0,0.0,@六小龄童,2019-02-16 20:20:38,2019-02-16,2019-02,5,20
1992,南洋术士,5.0,男,1.550319e+09,0.0,0.0,哈哈哈嗝,2019-02-16 20:14:56,2019-02-16,2019-02,5,20
1993,莫莫莫子若,6.0,女,1.550319e+09,0.0,0.0,哈哈哈哈哈哈哈哈哈哈哈,2019-02-16 20:10:09,2019-02-16,2019-02,5,20


In [3]:
# 按月份分组
y_month = cmt_df.groupby('month').count()['time']


x_month = list(y_month.reset_index()['month'])


y_month = list(i*9 for i in y_month)

print(x_month)
print(y_month)

['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-08']
[4959, 8802, 576, 441, 549, 225, 135, 63, 81, 432, 81, 36, 54, 63, 45, 54, 18, 9, 36, 396, 27, 27, 45, 45, 54, 36, 45, 153, 18, 63, 72, 81, 72, 72, 54, 9, 18, 9]


In [4]:
# 按照天数分组
y_day = cmt_df.groupby('day')['rcount'].sum()

x_day = list(y_day.reset_index()['day'])

y_day = list(y_day)


data_day = list(map(list,zip(x_day,y_day)))

print(data_day)

[['2019-01-19', 190.0], ['2019-01-20', 113.0], ['2019-01-21', 10.0], ['2019-01-22', 1.0], ['2019-01-26', 140.0], ['2019-01-27', 63.0], ['2019-01-28', 12.0], ['2019-01-29', 5.0], ['2019-01-30', 11.0], ['2019-01-31', 0.0], ['2019-02-01', 2.0], ['2019-02-02', 62.0], ['2019-02-03', 23.0], ['2019-02-04', 0.0], ['2019-02-05', 0.0], ['2019-02-06', 1.0], ['2019-02-07', 1.0], ['2019-02-08', 0.0], ['2019-02-09', 65.0], ['2019-02-10', 10.0], ['2019-02-11', 3.0], ['2019-02-12', 1.0], ['2019-02-13', 3.0], ['2019-02-14', 6.0], ['2019-02-15', 3.0], ['2019-02-16', 128.0], ['2019-02-17', 40.0], ['2019-02-18', 11.0], ['2019-02-19', 8.0], ['2019-02-20', 0.0], ['2019-02-21', 0.0], ['2019-02-22', 0.0], ['2019-02-23', 19.0], ['2019-02-24', 18.0], ['2019-02-25', 0.0], ['2019-02-26', 0.0], ['2019-02-27', 2.0], ['2019-02-28', 0.0], ['2019-03-02', 3.0], ['2019-03-03', 0.0], ['2019-03-04', 0.0], ['2019-03-05', 0.0], ['2019-03-07', 0.0], ['2019-03-08', 0.0], ['2019-03-09', 0.0], ['2019-03-10', 1.0], ['2019-03-11'

In [5]:
# 按照星期分组
x_week = ['星期一','星期二','星期三','星期四','星期五','星期六','星期七']
y_week = list(cmt_df.groupby('week').count()['time'])
y_week = list(i*9 for i in y_week)

dict1 = {
        "value": y_week[-1],
        "itemStyle": {
          "color": "#ff4500"
        }
}

dict2 = {
        "value": y_week[-2],
        "itemStyle": {
          "color": "#ff4500"
        }
}

# 将周六日特殊标记
y_week[-1] = dict1
y_week[-2] = dict2

y_week

[1557,
 1674,
 657,
 1026,
 729,
 {'value': 6471, 'itemStyle': {'color': '#ff4500'}},
 {'value': 5841, 'itemStyle': {'color': '#ff4500'}}]

In [6]:
# 按照时间分组
x_hour = list(range(24))
y_hour = cmt_df.groupby('hour').count()['time']
y_hour = list(i*9 for i in y_hour)

# y_hour

In [7]:
# 按照性别分组
x_sex = ['保密','女','男']
y_sex = cmt_df.groupby('sex').count()['time']
y_sex = list(i*9 for i in y_sex)

data_sex = [
    {"value":y_sex[0],'name':x_sex[0]},
    {"value":y_sex[1],'name':x_sex[1]},
    {"value":y_sex[2],'name':x_sex[2]},
]
    

data_sex

[{'value': 8694, 'name': '保密'},
 {'value': 2349, 'name': '女'},
 {'value': 6912, 'name': '男'}]

In [8]:
# 不同性别观看时间段
# 男
y_male_hour = cmt_df[cmt_df['sex'] == '男'].groupby('hour').count()['time']
y_male_hour = list(i*9 for i in y_male_hour)
# 女
y_female_hour = cmt_df[cmt_df['sex'] == '女'].groupby('hour').count()['time']
y_female_hour = list(i*9 for i in y_female_hour)

print(y_male_hour)
print(y_female_hour)

[387, 135, 153, 54, 27, 54, 27, 63, 108, 189, 90, 135, 216, 207, 297, 153, 225, 126, 270, 1458, 864, 603, 558, 513]
[90, 54, 63, 45, 9, 9, 27, 18, 63, 90, 36, 27, 72, 99, 108, 63, 90, 45, 108, 468, 234, 198, 198, 135]


In [9]:
# 按照等级分组
x_lv = [0,1,2,3,4,5,6]

y_lv =  cmt_df.groupby('level').count()['time']

y_lv.loc[1.0] = 5

y_lv = y_lv.sort_index()

y_lv = list(i*9 for i in y_lv)


data_lv=[
    list(cmt_df[cmt_df['level']==0][cmt_df['like']>1]['like'].map(lambda x:math.log(x,10))),
    [0.1,0.2,0.4],
    list(cmt_df[cmt_df['level']==2][cmt_df['like']>1]['like'].map(lambda x:math.log(x,10))),
    list(cmt_df[cmt_df['level']==3][cmt_df['like']>1]['like'].map(lambda x:math.log(x,10))),
    list(cmt_df[cmt_df['level']==4][cmt_df['like']>1]['like'].map(lambda x:math.log(x,10))),
    list(cmt_df[cmt_df['level']==5][cmt_df['like']>1]['like'].map(lambda x:math.log(x,10))),
    list(cmt_df[cmt_df['level']==6][cmt_df['like']>1]['like'].map(lambda x:math.log(x,10))),
]

data_lv

/Users/soutsukyou/PyCharm_Workspace/VSAS/venv/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/soutsukyou/PyCharm_Workspace/VSAS/venv/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/soutsukyou/PyCharm_Workspace/VSAS/venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Users/soutsukyou/PyCharm_Workspace/VSAS/venv/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/soutsukyou/PyCharm_Workspace/VSAS/venv/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/soutsukyou/PyCharm_Workspace/VSAS/venv/lib/python3.7/site-packages/ipykernel_la

[[0.6020599913279623,
  0.30102999566398114,
  0.30102999566398114,
  0.6020599913279623],
 [0.1, 0.2, 0.4],
 [0.7781512503836435, 0.30102999566398114],
 [0.30102999566398114,
  0.6020599913279623,
  0.47712125471966244,
  0.30102999566398114,
  0.7781512503836435],
 [2.4183012913197452,
  1.724275869600789,
  0.7781512503836435,
  0.47712125471966244,
  0.47712125471966244,
  0.30102999566398114,
  0.30102999566398114,
  0.30102999566398114,
  0.30102999566398114,
  0.30102999566398114,
  0.30102999566398114,
  0.30102999566398114,
  0.30102999566398114,
  1.7634279935629371,
  1.2041199826559246,
  0.47712125471966244,
  0.47712125471966244,
  0.47712125471966244,
  0.30102999566398114,
  0.47712125471966244,
  0.30102999566398114,
  2.305351369446624,
  0.47712125471966244,
  0.30102999566398114,
  0.30102999566398114,
  1.0],
 [4.1126720171171325,
  2.580924975675619,
  2.3180633349627615,
  1.845098040014257,
  0.8450980400142567,
  1.57978359661681,
  1.6627578316815739,
  0.9542

In [10]:
# 前10热评
tmp_df = cmt_df.sort_values(by='like',ascending=False).head(10)

x_comment_10 = list(tmp_df['content'])
y_comment_10 = list(tmp_df['like'])

data_comment_10 = list()

for i in range(10):
    data_comment_10.append({'value':y_comment_10[i],'name':x_comment_10[i]})

print(data_comment_10)

[{'value': 12962.0, 'name': '传统媒体拥抱年轻人的成功案例(°∀°)ﾉ'}, {'value': 8239.0, 'name': '未来b站的最强综艺即将诞生，我们都是见证者，哈哈[2233娘_大笑]'}, {'value': 5872.0, 'name': '笑死我了，太皮了。(°∀°)ﾉ\n支持b站最强综艺！'}, {'value': 4951.0, 'name': '电视在放哈哈哈哈哈哈天嘞'}, {'value': 4541.0, 'name': '你找孙经理关我孙师傅什么事情[蛆音娘_滑稽]'}, {'value': 3143.0, 'name': '配音小哥笑场了！扣钱！[蛆音娘_滑稽]'}, {'value': 3027.0, 'name': 'B站最强综艺[小电视_赞]二月霸权！[2233娘_大笑]'}, {'value': 2155.0, 'name': '我的快乐源泉[2233娘_喝水]'}, {'value': 1987.0, 'name': '随着节目的期数增长剪辑越来越皮了，干脆叫1818皮皮眼[蛆音娘_滑稽]'}, {'value': 1931.0, 'name': '“刚在采访的时候，门口还有小朋友问我，小八叔叔，你在浙江采访路上到底有多少富婆朋友啊？”'}]


In [11]:
# 按等级分组的热评
y_like_lv = cmt_df.groupby('level')['like'].mean()
y_like_lv.loc[1.0] = 1.010214
y_like_lv = round(y_like_lv,2)
y_like_lv = list(y_like_lv.sort_index())

y_like_lv

[1.07, 1.01, 1.14, 0.93, 7.38, 25.76, 49.81]

In [12]:
# 词云（后续可以通过编写中文词统计来替换）
wordcloud_data = [
        {
            "name": "哈哈哈",
            "value": 1446
        },
        {
            "name": "黄金眼",
            "value": 989
        },
        {
            "name": "1818",
            "value": 977
        },
        {
            "name": "灵魂拷问",
            "value": 1145
        },
        {
            "name": "沙雕",
            "value": 856
        },
        {
            "name": "电视",
            "value": 525
        },
        {
            "name": "记者",
            "value": 623
        },
        {
            "name": "消费者",
            "value": 243
        },
        {
            "name": "快乐源泉",
            "value": 122
        },
        {
            "name": "笑场",
            "value": 154
        },
        {
            "name": "好喜欢",
            "value": 351
        },
        {
            "name": "挨揍",
            "value": 99
        },
        {
            'name': '(〜￣△￣)〜',
            'value': 88
        },
        {
            'name': '笑死我了',
            'value': 492
        },
        {
            'name': '最强综艺',
            'value': 23
        },
        {
            'name': '连续剧',
            'value': 78
        },
        {
            'name': '关注',
            'value': 34
        },
        {
            'name': '战术回避',
            'value': 391
        },
        {
            'name': '笑死',
            'value': 45
        },
        {
            'name': '量子力学',
            'value': 213
        },
        {
            'name': '骗子',
            'value': 41
        },
        {
            'name': '薛定谔',
            'value': 108
        },
        {
            'name': '新闻',
            'value': 554
        },
        {
            'name': '有毒',
            'value': 48
        },
        {
            'name': '奇葩',
            'value': 61
        },
        {
            'name': '不够看',
            'value': 97
        },
        {
            'name': '打卡',
            'value': 124
        },
        {
            'name': '人才',
            'value': 22
        },
        {
            'name': '停不下来',
            'value': 412
        },
        {
            'name': '小吴',
            'value': 10
        },
        {
            'name': '鞭尸',
            'value': 24
        },
        {
            'name': '名场面',
            'value': 201
        },
        {
            'name': 'xswl',
            'value': 112
        },
        {
            'name': '玩梗',
            'value': 52
        },
        {
            'name': '无中生友',
            'value': 89
        },
]

In [13]:
# 热力图数据

y_hot = [0,0.125,0.25,0.375,0.5,0.625,0.75,0.875,1]
x_hot = ['1818黄金眼','店家','记者','灵魂拷问','遇事不决','骗子','战术回避','量子力学']

hot_value = [
    [1,0.32,0.71,0.4,0.21,0.03,0.11,0.09],
    [0.32,1,0.81,0.33,0.82,0.91,0.89,0.77],
    [0.77,0.65,1,0.93,0.2,0.01,0.13,0.11],
    [0.46,0.45,0.79,1,0.21,0.31,0.21,0.13],
    [0.05,0.51,0.10,0.24,1,0.33,0.43,0.81],
    [0.09,0.73,0.13,0.34,0.45,1,0.54,0.61],
    [0.23,0.55,0.08,0.21,0.55,0.64,1,0.45],
    [0.21,0.61,0.11,0.34,0.81,0.63,0.61,1],
]

data_hot = []
for i in range(len(hot_value)):
    for j in range(len(hot_value[i])):
        data_hot.append([i,j,hot_value[i][j]])

## 情感分析

In [14]:
# 导入必要的包
from snownlp import SnowNLP

In [15]:
# 测试
text = '我真的谢谢你，管好你自己。'
print(SnowNLP(text).sentiments)


0.7821126233928855


In [31]:
# 评论
cmt_list = list(cmt_df['content'])
print(cmt_list)
cmt_score_df = pd.DataFrame([SnowNLP(str(x)).sentiments for x in cmt_list],columns=['score'])
print(cmt_score_df)

# 弹幕
print(brg_list)
brg_score_df = pd.DataFrame([SnowNLP(str(x)).sentiments for x in brg_list],columns=['score'])
print(brg_score_df)


['传统媒体拥抱年轻人的成功案例(°∀°)ﾉ', '电视在放哈哈哈哈哈哈天嘞', '配音小哥笑场了！扣钱！[蛆音娘_滑稽]', '我的快乐源泉[2233娘_喝水]', '好想上一次1818黄金眼！', '全国都在关注浙江人民日常生活2333', '未来b站的最强综艺即将诞生，我们都是见证者，哈哈[2233娘_大笑]', '我觉得B站的用户不是不喜欢看新闻，而是懒得去别的地方看。。。', '我一个广东人，广东的新闻没看，光顾着关心你们浙江人的生活了，太膨胀了我。', '不喜欢看浙江卫视的综艺节目，什么跑男，什么演员的诞生，矫揉造作，都太假。就爱看你们的6频道，1818黄金眼，钱塘老娘舅，相亲才会赢，这三档节目才好看。', '要不是这是个老牌综艺节目，我百分百会认为这是个营销号 求置顶', '看着好笑，其实出了事不好笑的，记者小伙伴们辛苦了…鞠躬！\n黄金眼之所以会红，是因为他（她）们有恒心，有毅力，有一颗寻求真相和维护正义的…emm怎么说着说着赶脚像是再说超级英雄？\n不错！1818就是广大小伙伴们的英雄！\n1818加油！\n\n这种“管闲事”的栏目各个电视台不少，来自各个方面的鸭梨和威胁之类的也肯定很多！毕竟容易得罪人嘛…\n唉～有不少这样的节目，“昙花一现”或者播着播着就“变味儿”，1818能始终如一，这应该就是大家爱1818的重要因素之一吧…', '我最喜欢的还是那句：有在床上一起休息过吗。', '出现了，大型鬼畜素材稳定供应商，没有中间商（搬运工）赚差价[小电视_赞]', '话说我今年看1818时间最长，我自己都惊呆了[2233娘_吃惊]', '寒假在家无聊？可以兼职做复读机，跟我一样只需要手机就能做，复制粘贴即可，每天收入0元，我身边的朋友都在做。', '对于某人打码的请求，我们拒绝了(°∀°)ﾉ', '都放到网络上来播了，就不要用电视上的画质了吧……帧数和电视上一样可以理解，但是……\n1）视频格式：网络时代就别管电视上的标准了，1080P，码率平均6Mbps，最大24Mbps，走起！要啥4:3？（720x576？？？1.9Mbps？？？）\n2）电脑屏幕就不要拿着摄像机拍了，直接录屏吧；\n3）评论截图注意一下网页的背景和视频的背景，背景颜色不一样看的很不舒服，可以把评论也做成MG动画；\n4）截图直接这样放大会有马赛克的现象，如果需要放大还请截图之前

In [36]:
import random
random.seed(2022)

wordcloud_feel = list()

for b in random.sample(brg_list,100):
    wordcloud_feel.append(
        {
            'name':b,
            'value':100* SnowNLP(b).sentiments
        }
    )

print(wordcloud_feel)

[{'name': '你再骂', 'value': 50.0}, {'name': '哈哈哈哈哈哈哈哈哈', 'value': 99.57269865929867}, {'name': '哈哈哈哈哈哈', 'value': 97.51349273004098}, {'name': '哈哈哈哈哈哈', 'value': 97.51349273004098}, {'name': '哈哈哈哈哈哈哈哈哈哈', 'value': 99.57269865929867}, {'name': '哈哈哈哈哈哈哈哈哈哈', 'value': 99.57269865929867}, {'name': '哈哈哈哈哈哈哈', 'value': 97.51349273004098}, {'name': '不用怀疑  这就是', 'value': 18.474046404007982}, {'name': '牛批了 哈哈哈', 'value': 86.63872640818212}, {'name': '哈哈哈哈哈哈哈哈哈哈', 'value': 99.57269865929867}, {'name': '曾经我以为我只会追电视剧，想不到追到节目领域来了', 'value': 98.78649852121679}, {'name': '搭卡 口头挖露', 'value': 44.7678458509865}, {'name': '泡利，波尔：你可拉倒吧', 'value': 30.401091203896826}, {'name': '岂不妙哉！小伙子', 'value': 23.765728113198925}, {'name': '违规了，雨遮不能装的', 'value': 24.870834163912036}, {'name': '哈哈哈哈哈哈哈哈思路清晰', 'value': 99.73743738844176}, {'name': '这是捅了姓林的窝了？林校长？林先生？', 'value': 99.1964991574122}, {'name': '体验撞车的感觉', 'value': 31.02452320391015}, {'name': '祝师傅平安吧', 'value': 20.465382221314755}, {'name': '哈哈哈哈智慧的眼神有没有！！！', 'va

In [20]:
# 等到区间统计
x_score = list(range(8))
x_score_label = ['0～0.125','0.125～0.25','0.25～0.375','0.375～0.5','0.5～0.625','0.62～0.75','0.75～0.875','0.875～1']

cmt_score_list = []
for i in range(8):
    cmt_score_list.append(list(cmt_score_df[(cmt_score_df['score']>i*0.125)&(cmt_score_df['score']<=(i+1)*0.125)]['score']))

cmt_score_count_list = []
for i in range(8):
    cmt_score_count_list.append(len(cmt_score_list[i]))

cmt_score_count_list = [9*x for x in cmt_score_count_list]

print(cmt_score_count_list)


brg_score_list=[]
for i in range(8):
    brg_score_list.append(list(brg_score_df[(brg_score_df['score']>i*0.125)&(brg_score_df['score']<=(i+1)*0.125)]['score']))


brg_score_count_list = []
for i in range(8):
    brg_score_count_list.append(len(brg_score_list[i]))

print(brg_score_count_list)

[1773, 1125, 1377, 2439, 1881, 1593, 2151, 5616]
[957, 1201, 1253, 3520, 1776, 1513, 2444, 4150]


In [28]:
x_score_5 = list(range(30))
y_score_5 = list(random.uniform(0.2,0.8) for i in range(30))

## 数据汇总为json文件


In [37]:
dict_data = {
    'p1':{
        'x_day': x_day,
        'y_day': y_day,
        'data_day': data_day,
        'x_month':x_month,
        'y_month':y_month,
        'x_week': x_week,
        'y_week': y_week,
        'x_hour': x_hour,
        'y_hour': y_hour,
        'x_sex': x_sex,
        'y_sex': y_sex,
        'data_sex':data_sex,
        'y_male_hour':y_male_hour,
        'y_female_hour':y_female_hour,
        'x_lv': x_lv,
        'y_lv': y_lv,
        'data_lv': data_lv,
        'y_like_lv': y_like_lv,
        'x_comment_10': x_comment_10,
        'y_comment_10': y_comment_10,
        'data_comment_10': data_comment_10,
        'wordcloud_data': wordcloud_data,
        'x_hot':x_hot,
        'y_hot':y_hot,
        'data_hot':data_hot
    },
    'p2': {
        'wordcloud_feel': wordcloud_feel,
        'x_score': x_score,
        'x_score_5':x_score_5,
        'y_score_5':y_score_5,
        'x_score_label': x_score_label,
        'cmt_score': {
            'count': cmt_score_count_list
        },
        'brg_score':{
            'count': brg_score_count_list
        }
    }
}

In [38]:
json_data = json.dumps(dict_data,ensure_ascii=False)

with open('../json_data.json','w') as f:
    f.write(json_data)

json_data

'{"p1": {"x_day": ["2019-01-19", "2019-01-20", "2019-01-21", "2019-01-22", "2019-01-26", "2019-01-27", "2019-01-28", "2019-01-29", "2019-01-30", "2019-01-31", "2019-02-01", "2019-02-02", "2019-02-03", "2019-02-04", "2019-02-05", "2019-02-06", "2019-02-07", "2019-02-08", "2019-02-09", "2019-02-10", "2019-02-11", "2019-02-12", "2019-02-13", "2019-02-14", "2019-02-15", "2019-02-16", "2019-02-17", "2019-02-18", "2019-02-19", "2019-02-20", "2019-02-21", "2019-02-22", "2019-02-23", "2019-02-24", "2019-02-25", "2019-02-26", "2019-02-27", "2019-02-28", "2019-03-02", "2019-03-03", "2019-03-04", "2019-03-05", "2019-03-07", "2019-03-08", "2019-03-09", "2019-03-10", "2019-03-11", "2019-03-12", "2019-03-13", "2019-03-16", "2019-03-17", "2019-03-20", "2019-03-22", "2019-03-23", "2019-03-24", "2019-03-25", "2019-03-28", "2019-04-01", "2019-04-05", "2019-04-07", "2019-04-08", "2019-04-11", "2019-04-15", "2019-04-16", "2019-04-17", "2019-04-19", "2019-04-20", "2019-04-21", "2019-04-22", "2019-04-23", "